In [20]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
import codecs
import os
import re

BALA_TRAIN='original_data/bala_training.utf8'
BALA_TEST='original_data/bala_training.utf8'

CHINESE_IDIOMS='original_data/idioms'
OUTPUT_PATH='data'

rNUM = '(-|\+)?\d+((\.|·)\d+)?%?'
rENG = '[A-Za-z_.]+'
START='S'
END='E'


In [21]:
def strQ2B(ustring):
    """全角转半角"""
    rstring = ""
    for uchar in ustring:
        inside_code = ord(uchar)
        if inside_code == 12288:  # 全角空格直接转换
            inside_code = 32
        elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
            inside_code -= 65248
        #rstring += unichr(inside_code)
        rstring += chr(inside_code)
        
    return rstring


def preprocess(input,output):
    ''' pku_training.utf8(input) 生成一个pku_train_all(output)文件 '''
    output_filename = os.path.join(OUTPUT_PATH,output)
    idioms=dict()
    #取出成语词典中的成语
    with codecs.open(CHINESE_IDIOMS,'r','utf-8') as f:    
        for line in f:  
            idioms[line.strip()]=1    #返回移除字符串头尾指定的字符(空格)生成的新字符串                              标记为1
    count_idioms = 0
    sents=[]
    #pku_training.utf8的文本处理：
    with codecs.open(input,'r','utf-8') as fin:
        with codecs.open(output_filename,'w','utf-8') as fout:
            for line in fin:   #取pku_training 的每一行
                sent=strQ2B(line).split( ) #此行 词 的集合
                new_sent=[]
                for word in sent:
                    word=re.sub(rNUM,'0',word)  #正则 没懂
                    word=re.sub(rENG,'X',word)
                    if idioms.get(word) is not None:
                        count_idioms+=1
                        word=u'I'
                    new_sent.append(word)
                sents.append(new_sent)
            for sent in sents:
                fout.write('  '.join(sent))
                fout.write('\n')
    #print 'idioms count:%d' % count_idioms

def split(dataset): #划分出验证集
#dataset='pku'
    dataset=os.path.join(OUTPUT_PATH,dataset)
    with codecs.open(dataset+'_train_all','r','utf-8') as f:
	    lines = f.readlines()
	    idx = int(len(lines)*0.9)
	    with codecs.open(dataset+'_train','wb','utf-8') as fo:
		    for line in lines[:idx]:
			    fo.write(line.strip()+'\r')
	    with codecs.open(dataset+'_dev','wb','utf-8') as fo:
		    for line in lines[idx:]:
			    fo.write(line.strip()+'\r')
    os.remove(dataset+'_train_all')


def ngram(ustr,n=2):
    ngram_list=[]
    for i in range(len(ustr)-n+1):
        ngram_list.append(ustr[i:i+n])
    return ngram_list

def bigram_words(dataset,window_size=2):
#dataset=‘pku_train’
    dataset=os.path.join(OUTPUT_PATH,dataset)
    words=dict()   
    start=''.join([START]*window_size)   # start=‘SS’
    end=''.join([END]*window_size)      
    with codecs.open(dataset,'r','utf-8') as f:
        for line in f:
            line=start+re.sub('\s+','',line.strip())+end
            for word in ngram(line,window_size):
                words[word]=words.get(word,0)+1
    with codecs.open(dataset+'_bigram','w','utf-8') as f:
        for k,v in words.items():
            f.write(k+' '+str(v)+'\n')



In [22]:
if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)
#preprocess
print("start preprocess")

preprocess(BALA_TRAIN,'bala_train_all') #pku_training.utf8 生成一个pku_train_all文件
preprocess(BALA_TEST,'bala_test')


#split
print("start split")
split('bala')   #划分出验证集,pku_train_all变为pku_train和pku_dev

#bigram
print("start bigram")
bigram_words('bala_train')


start preprocess
start split
start bigram
